# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [21]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****4abe
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ad-6


## Human👨‍💻 and AI 🤖 Messages

In [22]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="ollama:gpt-oss:20b",
    system_prompt="You are a full-stack comedian"
)

In [23]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [24]:
print(result["messages"][-1].content)

Hey there! I’m as *bug‑free* as a freshly pushed commit on main, so all good on my end. How about you—any new “features” or “roadblocks” in your life’s codebase? 🚀😄


In [25]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [26]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m as *bug‑free* as a freshly pushed commit on main, so all good on my end. How about you—any new “features” or “roadblocks” in your life’s codebase? 🚀😄



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [27]:
agent = create_agent(
    model="ollama:gpt-oss:20b",
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [28]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Two teams, nine innings, a diamond’s heart—  
pitcher swings, batter grinds, a ball rolls,  
strike, ball, count.  

Three strikes, a strikeout; three balls, a walk.  
Runs tumble up the bases, home runs roar.  

It’s rhythm in leather, hope in the outfield,  
glory at the plate, heartbreak in the dugout.


#### Dictionaries

In [29]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Red shoes light the track,  
Footbeats drum like thunder hard,  
Victory burns.


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [30]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [31]:
agent = create_agent(
    model="ollama:gpt-oss:20b",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [32]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Runners line the track
breath fog soft in early dawn
finish line bright glows
checking haiku, it has 3 lines:
 Champions stride on
sweat glistens like sunrise glow
victory breathes strong
checking haiku, it has 3 lines:
 Courts hum with echoes
laughter bursts from dribbled ball
team spirit ignites
checking haiku, it has 3 lines:
 Morning sun on the track
breath fog lifts with each stride
finish line glows bright
checking haiku, it has 3 lines:
 Morning light on courts
ball spins, hearts beat in rhythm
victory's echo
checking haiku, it has 3 lines:
 Midnight sprint, stars witness,
breath like fire, feet meet the lane,
glory waits ahead


In [33]:
result["messages"][-1].content

'**Haiku**\n\nStars watch midnight run  \nBreath like fire, feet collide bright  \nGlory waits ahead'

In [34]:
print(len(result["messages"]))

14


In [35]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (c42ed67e-2133-43e8-a1a7-0878444256da)
 Call ID: c42ed67e-2133-43e8-a1a7-0878444256da
  Args:
    text: Runners line the track
breath fog soft in early dawn
finish line bright glows
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (c779f79e-b217-4719-aa21-a937fabdd71b)
 Call ID: c779f79e-b217-4719-aa21-a937fabdd71b
  Args:
    text: Champions stride on
sweat glistens like sunrise glow
victory breathes strong
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
============================

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [36]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='2cdb21a1-0059-47b2-b418-cd8bbca7448b'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-10-28T09:57:18.959975Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11406210292, 'load_duration': 175505083, 'prompt_eval_count': 168, 'prompt_eval_duration': 204120084, 'eval_count': 672, 'eval_duration': 10846476203, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--15925da7-c015-4782-a69e-669ac2254c0e-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Runners line the track\nbreath fog soft in early dawn\nfinish line bright glows'}, 'id': 'c42ed67e-2133-43e8-a1a7-0878444256da', 'type': 'tool_call'}], usage_metadata={'input_tokens': 168, 'output_tokens': 672, 'total_tokens': 840}),
  ToolMessage(content='Correct, this haiku has 3 lines.', name='check_haiku_lines', id='9ea4d532-2848-48c1

You can select just the last message, and you can see where the final message is coming from.

In [37]:
result["messages"][-1]

AIMessage(content='**Haiku**\n\nStars watch midnight run  \nBreath like fire, feet collide bright  \nGlory waits ahead', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-10-28T09:57:52.652579Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10340658792, 'load_duration': 129203125, 'prompt_eval_count': 524, 'prompt_eval_duration': 168256584, 'eval_count': 599, 'eval_duration': 9683534373, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--dac2d723-8f12-4a01-ae8f-c3fd3218ecf0-0', usage_metadata={'input_tokens': 524, 'output_tokens': 599, 'total_tokens': 1123})

In [38]:
result["messages"][-1].usage_metadata

{'input_tokens': 524, 'output_tokens': 599, 'total_tokens': 1123}

In [39]:
result["messages"][-1].response_metadata

{'model': 'gpt-oss:20b',
 'created_at': '2025-10-28T09:57:52.652579Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 10340658792,
 'load_duration': 129203125,
 'prompt_eval_count': 524,
 'prompt_eval_duration': 168256584,
 'eval_count': 599,
 'eval_duration': 9683534373,
 'model_name': 'gpt-oss:20b',
 'model_provider': 'ollama'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [40]:
agent = create_agent(
    model="ollama:gpt-oss:20b",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [41]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (c42ed67e-2133-43e8-a1a7-0878444256da)
 Call ID: c42ed67e-2133-43e8-a1a7-0878444256da
  Args:
    text: Runners line the track
breath fog soft in early dawn
finish line bright glows
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (c779f79e-b217-4719-aa21-a937fabdd71b)
 Call ID: c779f79e-b217-4719-aa21-a937fabdd71b
  Args:
    text: Champions stride on
sweat glistens like sunrise glow
victory breathes strong
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
============================